In [48]:
import random
import math

In [114]:
#graf je predstavljen kao mapa
class Graph:
    #ajdacency_list - mapira cvorove u listu cvorova sa kojima je povezan
    #cost - niz cena cvorova
    #V - broj cvorova
    def __init__(self, adjacency_list, cost):
        self.cost = cost
        self.adj = adjacency_list
        self.V = len(self.adj)
    
    #DFS obilazak grafa
    #povratna vrednost - lista cvorova
    def DFSUtil(self, temp, node, visited):
        visited[node] = True
        temp.append(node)
        for v in self.adj.get(node):
            if visited[v] == False:
                temp = self.DFSUtil(temp, v, visited)
        return temp
    
    #povratna vreednost - lista komponenti
    def connectedComponents(self):
        visited = {}
        cc = []
        for k in self.adj.keys() :
            visited[k] = False
        for (k,v) in self.adj.items():
            if visited[k] == False:
                temp = []
                cc.append(self.DFSUtil(temp, k, visited))
        return cc
    #funkcija vraca novi graf koji ne sadrzi cvor node (kao ni njegove grane)
    #izbacujemo cvor i sve grane koje polaze iz tog cvora
    def remove_node(self,node):
        newAdj = {}
        for (k,v) in self.adj.items():
            if k == node:
                continue
            else:
                newAdj[k] = []
                for i in range(len(v)):
                    if v[i] != node:
                        newAdj[k].append(v[i])
                        
        return Graph(newAdj, self.cost)

    #povratna vrednost funkcije je najveca cena cvora iz datog grafa
    def maxCost(self):
        maks = -float('inf')
        #prolazimo kroz sve cvovore koji se nalaze u grafu
        for (k,v) in self.adj.items():
            if self.cost[k] > maks:
                maks = self.cost[k]
        return maks

In [122]:
adjacency_list = {0:[1,2,5], 1:[0,5,2],2:[0,1,3,5], 3:[2,4], 4:[3,5],5:[0,1,2,4]}
vertics_cost = {0:8,1:4,2:1,3:6,4:5,5:4}
graph = Graph(adjacency_list, vertics_cost)
print(graph.adj)
print(graph.connectedComponents())
graphWithout0 = graph.remove_node(2)
print(graphWithout0.adj)
print(graphWithout0.connectedComponents())
print(graphWithout0.maxCost())
graphWithout02 = graphWithout0.remove_node(4)
print(graphWithout02.adj)
print(graphWithout02.connectedComponents())
print(graphWithout02.maxCost())

{0: [1, 2, 5], 1: [0, 5, 2], 2: [0, 1, 3, 5], 3: [2, 4], 4: [3, 5], 5: [0, 1, 2, 4]}
[[0, 1, 5, 2, 3, 4]]
{0: [1, 5], 1: [0, 5], 3: [4], 4: [3, 5], 5: [0, 1, 4]}
[[0, 1, 5, 4, 3]]
8
{0: [1, 5], 1: [0, 5], 3: [], 5: [0, 1]}
[[0, 1, 5], [3]]
8


In [127]:
#klasa jedinki
#prosledjujemo joj pocetni graf
#jedinka je reprezentovana u obliku niza nula i jedinica (niz je velicine broja cvorova u grafu)
#nula oznacava da se cvor (indeks niza oznacava cvor) nalazi u jednom podgrafu
#a jedinica da se naalzi u drugom
class Individual:
    def __init__(self, graph):
        self.graph = graph
        self.code = []
        self.V = len(graph.adj)
        for i in range(self.V):
            self.code.append(random.random() < 0.5)
        self.fitness = self.fitnessFunction()
    #definise nacin poredjenja jedinki
    def __lt__(self, other):
        return self.fitness < other.fitness
    #racunamo u odnosu na broj komponenti podgrafova
    def penalty_function(self):
        graph1, graph2 = self.getSubgraphs()
        self.subgraph1 = graph1
        self.subgraph2 = graph2
        ncc1 = len(graph1.connectedComponents())
        ncc2 = len(graph2.connectedComponents())
                        
        max1 = self.subgraph1.maxCost()
        max2 = self.subgraph2.maxCost()
      
        return (ncc1-1)*max1 + (ncc2-1)*max2

    #fitnes raacuna koliko su priblizne vrednosti suma dva podgrafa
    #sto je fitnes manji resenje je bolje
    #penalti funcija sluzi kao kazneni poeni ako dobijemo podgrafove
    #koji nisu povezani
    def fitnessFunction(self):
        w1 = 0
        w2 = 0
        for i in range(self.V):
            if self.code[i]:
                w1 += self.graph.cost[i]
            else:
                w2 += self.graph.cost[i]

        return abs(w1-w2) + self.penalty_function()
    
    def getSubgraphs(self):
        graph1 = Graph(self.graph.adj.copy(), self.graph.cost)
        graph2 = Graph(self.graph.adj.copy(), self.graph.cost)
        for i in range(self.V):
            if self.code[i]:
                graph2 = graph2.remove_node(i)
            else:
                graph1 = graph1.remove_node(i)
        return graph1, graph2


In [128]:
#provera klase Individual
ind = Individual(graph)
print(ind.graph.adj)
print(ind.subgraph1.adj)
print(ind.subgraph2.adj)
print(ind.code)
print(ind.fitness)


{0: [1, 2, 5], 1: [0, 5, 2], 2: [0, 1, 3, 5], 3: [2, 4], 4: [3, 5], 5: [0, 1, 2, 4]}
{0: [], 3: []}
{1: [5, 2], 2: [1, 5], 4: [5], 5: [1, 2, 4]}
[True, False, False, True, False, False]
8


In [129]:
#turnir selekcija
# nTournament - velicina turnira
def selection(population):
    min = float('inf')
    for i in range(nTournament):
        j = random.randrange(len(population))
        if population[j].fitness < min:
            min = population[j].fitness
            k = j
    return k

In [130]:
def crossover(parent1, parent2, child1, child2):
    i = random.randrange(len(parent1.code))
    for j in range(i):
        child1.code[j] = parent1.code[j]
        child2.code[j] = parent2.code[j]
    for j in range(i, len(parent1.code)):
        child1.code[j] = parent2.code[j]
        child2.code[j] = parent1.code[j]

In [131]:
#mutacija
#probability - verovatnoca mutacije na genu
def mutation(individual):
    for i in range(len(individual.code)):
        if random.random() > probability:
            continue
        individual.code[i] = not individual.code[i]

In [132]:
#parametri genetskog algoritma
nTournament = 6
probability = 0.05
nPopulation = 100
nIterations = 500
nElite = 30 

In [133]:
#genetski algoritam
def genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph):
    population = []
    newPopulation = []
    for i in range(nPopulation):
        population.append(Individual(graph))
        newPopulation.append(Individual(graph))

    for iteration in range(nIterations):
        population.sort()
        #sacuvam elitne jedinke - prvih nElite jedinki ostaju u populaciji
        for i in range(nElite):
            newPopulation[i] = population[i]

        for i in range(nElite, nPopulation, 2):
            k1 = selection(population)
            k2 = selection(population)
            crossover(population[k1], population[k2],newPopulation[i],newPopulation[i+1])
            mutation(newPopulation[i])
            mutation(newPopulation[i+1])
            newPopulation[i].fitness = newPopulation[i].fitnessFunction()
            newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction()

        population = newPopulation

    population.sort()
    return population[0]

In [134]:
def print_solution(solution):
    print('Solution: ')
    print('Graph1 - ', solution.subgraph1.adj)
    print('Graph2 - ', solution.subgraph2.adj)
    print('Fitness - ', solution.fitness)
    print('Code - ', solution.code)

In [135]:
adjacency_list = {0:[1,2,5], 1:[0,5,2],2:[0,1,3,5], 3:[2,4], 4:[3,5],5:[0,1,2,4]}
vertics_cost = [8,4,1,6,5,4]
graph = Graph(adjacency_list, vertics_cost)
solution = genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph)
print_solution(solution)


Solution: 
Graph1 -  {0: [1, 2], 1: [0, 2], 2: [0, 1]}
Graph2 -  {3: [4], 4: [3, 5], 5: [4]}
Fitness -  2
Code -  [True, True, True, False, False, False]


In [120]:
adjacency_list = {0:[1,2], 1:[0,4],2:[0,3],3:[2,4], 4:[1,3]}
vertics_cost = [3,7,1,4,9]
graph = Graph(adjacency_list, vertics_cost)
solution = genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph)
print_solution(solution)

Solution: 
Graph1 -  {0: [1, 2], 1: [0], 2: [0]}
Graph2 -  {3: [4], 4: [3]}
Fitness -  2
Code -  [True, True, True, False, False]


In [121]:
vertics_cost = [1,2,3,4,5,6,6,5,4,1,2,3,3,2,1,2,5,4,3,2,4,3,2,4,2,1,3,1,1,1,3,1]
adjacency_list = {0:[1,3],1:[0,3,20,11,6,28],2:[9,10,24],3:[0,1,9,12,4,5],4:[3,19,5],5:[3,4], 6:[1,7,8], 7:[6,8,14],8:[6,7,9], 9:[8,20,3,2,10,12,30,23,16,29,15],10:[2,17,9],11:[1,13,18,25,12],12:[11,3,9],13:[18,11],14:[7,15,27],15:[14,12],16:[22,23,17,9],17:[16,10],18:[19,11,27,13],19:[18,27,4,31,26],20:[1,9],21:[28,22],22:[29,21,16],23:[16,9,24,30],24:[23,31,2],25:[28,11,29],26:[19,31],27:[14,18,19],28:[21,1,25],29:[25,22,9],30:[23,9],31:[19,26,24]}
graph = Graph(adjacency_list, vertics_cost)
solution = genetic_algorithm(nPopulation, nIterations, nElite, nTournament, probability, graph)
print_solution(solution)

Solution: 
Graph1 -  {0: [3], 2: [9, 10], 3: [0, 9, 4, 5], 4: [3, 19, 5], 5: [3, 4], 8: [9], 9: [8, 20, 3, 2, 10, 30, 29], 10: [2, 9], 11: [13, 18, 25], 13: [18, 11], 18: [19, 11, 13], 19: [18, 4], 20: [9], 25: [11, 29], 29: [25, 9], 30: [9]}
Graph2 -  {1: [6, 28], 6: [1, 7], 7: [6, 14], 12: [], 14: [7, 15, 27], 15: [14, 12], 16: [22, 23, 17], 17: [16], 21: [28, 22], 22: [21, 16], 23: [16, 24], 24: [23, 31], 26: [31], 27: [14], 28: [21, 1], 31: [26, 24]}
Fitness -  0
Code -  [True, False, True, True, True, True, False, False, True, True, True, True, False, True, False, False, False, False, True, True, True, False, False, False, False, True, False, False, False, True, True, False]
